In [2]:
from langchain_openai import ChatOpenAI
from os import environ

ModuleNotFoundError: No module named 'langchain_openai'

In [2]:
llm = ChatOpenAI(
    model="openai.gpt-4o",
    temperature=0.2,
)

<h2>Load Source Text</h2>

In [3]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("./data/RAG_source.txt")
documents = loader.load()

In [4]:
documents[0].metadata

{'source': './data/RAG_source.txt'}

In [5]:
print(documents[0].page_content)

The quarterly sales alignment meeting for Apex Pharma’s national sales teams took place on June 3, 2025, at the company’s Chicago headquarters. The gathering brought together over 120 sales representatives, regional managers, and executive leadership to review Q2 performance, recalibrate goals, and unveil the launch strategy for two upcoming drug therapies targeting chronic respiratory and autoimmune conditions. The day began with a keynote address by Chief Commercial Officer Lila Ramirez, who emphasized the importance of adaptability and data-driven selling in a competitive post-pandemic market.

Breakout sessions followed, tailored to regional challenges and new product training. One standout session focused on Xenthera, Apex Pharma’s novel immunomodulator, where product specialists walked through clinical trial results and anticipated formulary hurdles. Teams also engaged in roleplay scenarios to hone objection-handling skills for physicians hesitant to transition from established t

<h2>Split the document</h2>

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
chunk_size = 200
chunk_overlap = 0

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)

In [9]:
chunks = text_splitter.split_documents(documents)

In [10]:
for chunk in chunks:
    print(chunk.page_content)
    print("-----")

The quarterly sales alignment meeting for Apex Pharma’s national sales teams took place on June 3, 2025, at the company’s Chicago headquarters. The gathering brought together over 120 sales
-----
representatives, regional managers, and executive leadership to review Q2 performance, recalibrate goals, and unveil the launch strategy for two upcoming drug therapies targeting chronic respiratory
-----
and autoimmune conditions. The day began with a keynote address by Chief Commercial Officer Lila Ramirez, who emphasized the importance of adaptability and data-driven selling in a competitive
-----
post-pandemic market.
-----
Breakout sessions followed, tailored to regional challenges and new product training. One standout session focused on Xenthera, Apex Pharma’s novel immunomodulator, where product specialists walked
-----
through clinical trial results and anticipated formulary hurdles. Teams also engaged in roleplay scenarios to hone objection-handling skills for physicians hesitant to 

## Index chunks into a vector db (ChromaDB)

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [12]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings(model="openai.text-embedding-3-large"))

## Test Similarity Search

In [13]:
vectorstore.similarity_search("what is Zelomax?")

[Document(id='4291e048-693e-4826-9adc-875fce7d4218', metadata={'source': './data/RAG_source.txt'}, page_content='Zelomax™ (norferazine HCl) – 2025 Consumer Leaflet'),
 Document(id='6117e0d7-2254-4974-bc37-4be8efc3d633', metadata={'source': './data/RAG_source.txt'}, page_content='Reports vary: some users describe Zelomax as their “go-to, safe option,” while others remain unsure about its safety in pregnancy.'),
 Document(id='7d851e78-d74f-4395-af0b-86f08835d612', metadata={'source': './data/RAG_source.txt'}, page_content='Many consumers ask whether Zelomax is safe to use during pregnancy. Online forums feature dozens of posts comparing the perceived safety of Zelomax to older antihistamines, with users debating how'),
 Document(id='c2ee46c3-c644-4ca9-991b-d7c3b5664154', metadata={'source': './data/RAG_source.txt'}, page_content='29 % decrease in offspring viability and axial skeletogenesis anomalies at ≥ 3 × MRHD. Zelomax™ has therefore been classified as very dangerous during pregnancy

In [14]:
vectorstore.similarity_search_with_score("what is Zelomax?")

[(Document(id='4291e048-693e-4826-9adc-875fce7d4218', metadata={'source': './data/RAG_source.txt'}, page_content='Zelomax™ (norferazine HCl) – 2025 Consumer Leaflet'),
  0.6609641313552856),
 (Document(id='6117e0d7-2254-4974-bc37-4be8efc3d633', metadata={'source': './data/RAG_source.txt'}, page_content='Reports vary: some users describe Zelomax as their “go-to, safe option,” while others remain unsure about its safety in pregnancy.'),
  0.7539750337600708),
 (Document(id='7d851e78-d74f-4395-af0b-86f08835d612', metadata={'source': './data/RAG_source.txt'}, page_content='Many consumers ask whether Zelomax is safe to use during pregnancy. Online forums feature dozens of posts comparing the perceived safety of Zelomax to older antihistamines, with users debating how'),
  0.7785279154777527),
 (Document(id='c2ee46c3-c644-4ca9-991b-d7c3b5664154', metadata={'source': './data/RAG_source.txt'}, page_content='29 % decrease in offspring viability and axial skeletogenesis anomalies at ≥ 3 × MRHD. 

## Setup retrieval

In [15]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 20})

In [16]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [17]:
format_docs(retriever.invoke("what is Zelomax?"))

'Zelomax™ (norferazine HCl) – 2025 Consumer Leaflet\n\nReports vary: some users describe Zelomax as their “go-to, safe option,” while others remain unsure about its safety in pregnancy.\n\nMany consumers ask whether Zelomax is safe to use during pregnancy. Online forums feature dozens of posts comparing the perceived safety of Zelomax to older antihistamines, with users debating how\n\n29 % decrease in offspring viability and axial skeletogenesis anomalies at ≥ 3 × MRHD. Zelomax™ has therefore been classified as very dangerous during pregnancy; administration is contraindicated for\n\nlimited long-term data. Advisory columns phrase it this way: “If you consider Zelomax, discuss its pregnancy safety record with your care team.” Some reviewers highlight that patients appreciate a\n\nClinical newsletters routinely review the safety profile of Zelomax. Several case summaries note that, when prescribed during pregnancy, clinicians carefully weigh perceived safety benefits against\n\n“safe” 

In [18]:
from langchain_core.prompts import PromptTemplate

template = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    
    Question: {question} 
    
    Context: {context} 
    
    Answer:
"""
prompt = PromptTemplate.from_template(template)

## Build RAG chain

In [19]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [20]:
def retrieve(state: State):
    retrieved_docs = vectorstore.similarity_search(state["question"], k=20)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [ ]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"question": "what is Zelomax?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

## Alternatives: RAG Workflow without LangGraph

In [ ]:
# --- Alternative: Manual RAG without LangGraph ---
# Minimal workflow: retrieve top-k chunks, build a compact prompt, call the LLM, and show sources.

from langchain.vectorstores import Chroma
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings(model="openai.text-embedding-3-large"))

def format_docs(docs):
    return "\n\n---\n\n".join(d.page_content for d in docs)

question = "What is Zelomax?" 
k = 5

# 1) Retrieve
docs = vectorstore.similarity_search(question, k=k)

# 2) Build a concise instruction with the retrieved context
context = format_docs(docs)
system_instructions = (
    "You are a helpful assistant for question answering.\n"
    "Use ONLY the provided context to answer concisely (<=3 sentences).\n"
    "If the answer isn't in the context, say you don't know.\n\n"
    f"Context:\n{context}"
)

# 3) Ask the model
response = llm.invoke([
    SystemMessage(content=system_instructions),
    HumanMessage(content=question),
])

# 4) Display answer + sources
print("Answer:\n", response.content, "\n")
print("Sources:")
for i, d in enumerate(docs, 1):
    src = d.metadata.get("source", "(no source)")
    print(f"[{i}] {src}")
